# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from numpy import log as ln

In [2]:
df = pd.read_excel("All supergames.xlsx")

# Vector

In [3]:
# The vector for All data :
AD0, ADCC, ADCD, ADDC, ADDD = 0.069,0.21,0.167,0.097,0.024
CSG0, CSGCC, CSGCD, CSGDC, CSGDD = 0.647, 0.974, 0.371, 0.773, 0.088
DSG0, DSGCC, DSGCD, DSGDC, DSGDD = 0.598, 0.958, 0.276, 0.253, 0.061

# Function

In [4]:
def MLE_K3 (data,AD0, ADCC, ADCD, ADDC, ADDD ,CSG0, CSGCC, CSGCD, CSGDC, CSGDD ,DSG0, DSGCC, DSGCD, DSGDC, DSGDD):
    for i in range(data.shape[0]):
       
        if data.iloc[i]['round'] == 1:
            if data.iloc[i]['coop'] == 'coop':
                data.loc[i, "AD"] = abs(ln(AD0))
                data.loc[i, "CSG"] = abs(ln(CSG0))
                data.loc[i, "DSG"] = abs(ln(DSG0))
            else:
                data.loc[i, "AD"] = abs(ln(1-AD0))
                data.loc[i, "CSG"] = abs(ln(1-CSG0))
                data.loc[i, "DSG"] = abs(ln(1-DSG0))
         
        
        else:
       
            if (data.iloc[i-1]['ocoop'] == "coop" and  data.iloc[i-1]['coop'] == "coop"):
                if  data.iloc[i]['coop'] == "coop":
                    data.loc[i, "AD"] = abs(ln(ADCC))
                    data.loc[i, "CSG"] = abs(ln(CSGCC))
                    data.loc[i, "DSG"] = abs(ln(DSGCC))
                else:
                    data.loc[i, "AD"] = abs(ln(1-ADCC))
                    data.loc[i, "CSG"] = abs(ln(1-CSGCC))
                    data.loc[i, "DSG"] = abs(ln(1-DSGCC))


            
            elif (data.iloc[i-1]['ocoop'] == "defect" and  data.iloc[i-1]['coop'] == "coop"):
                if  data.iloc[i]['coop'] == "coop":
                    data.loc[i, "AD"] = abs(ln(ADCD))
                    data.loc[i, "CSG"] = abs(ln(CSGCD))
                    data.loc[i, "DSG"] = abs(ln(DSGCD))

                else:
                    data.loc[i, "AD"] = abs(ln(1-ADCD))
                    data.loc[i, "CSG"] = abs(ln(1-CSGCD))
                    data.loc[i, "DSG"] = abs(ln(1-DSGCD))

                    
            elif (data.iloc[i-1]['ocoop'] == "coop" and  data.iloc[i-1]['coop'] == "defect"):
                if  data.iloc[i]['coop'] == "coop":
                    data.loc[i, "AD"] = abs(ln(ADDC))
                    data.loc[i, "CSG"] = abs(ln(CSGDC))
                    data.loc[i, "DSG"] = abs(ln(DSGDC))
                else:
                    data.loc[i, "AD"] = abs(ln(1-ADDC))
                    data.loc[i, "CSG"] = abs(ln(1-CSGDC))
                    data.loc[i, "DSG"] = abs(ln(1-DSGDC))

                    
            else:
                if (data.iloc[i-1]['ocoop'] == "defect" and  data.iloc[i-1]['coop'] == "defect"):
                    if  data.iloc[i]['coop'] == "coop":
                        data.loc[i, "AD"] = abs(ln(ADDD))
                        data.loc[i, "CSG"] = abs(ln(CSGDD))
                        data.loc[i, "DSG"] = abs(ln(DSGDD))

                    else:
                        data.loc[i, "AD"] = abs(ln(1-ADDD))
                        data.loc[i, "CSG"] = abs(ln(1-CSGDD))
                        data.loc[i, "DSG"] = abs(ln(1-DSGDD))

                        
    return(df)

In [5]:
def MLE_calculate(data):
    table = pd.pivot_table(data, values=['AD',"CSG","DSG"], index=['ID'], aggfunc=np.sum)
    return (table)

In [6]:
def MLE_min (data):
    mle = sum(data.min(axis=1))
    data['min'] = data.min(axis=1)
    return (mle,data)

In [7]:
def Cluster (data):
    cluster = []
    for i in range (data.shape[0]):
        if data.iloc[i]['AD']== data.iloc[i]['min']:
            cluster.append(1)
        if data.iloc[i]['CSG']== data.iloc[i]['min']:
            cluster.append(2)
        if data.iloc[i]['DSG']== data.iloc[i]['min']:
            cluster.append(3)
    data['class'] = cluster
    return(data)


In [8]:
def vLookup (data,table):
    data = pd.merge(data, table['class'], on ='ID', how ='inner',)
    return (data)

In [9]:
def change_class (df1,df2,fit=0):
    for i in range (df2.shape[0]):
        if df2.iloc[i]['class']== df1.iloc[i]['class']:
            fit+=1
    return (fit/1734)

In [10]:
def N_class (data):
    table2 = pd.pivot_table(data, values=["class"], index=['ID'], aggfunc=np.mean)
    
    class1 = sum(table2['class'] == 1)
    class2 = sum(table2['class'] == 2)
    class3 = sum(table2['class'] == 3)
    return(class1,class2,class3)

In [11]:
def vector (data):
    table2 = pd.pivot_table(data, values=['roundone','aftercc','aftercd', 'afterdc', 'afterdd'],
                        index=["class"], aggfunc=np.mean)
    
    AD0, ADCC, ADCD, ADDC, ADDD = table2.iloc[0,4],table2.iloc[0,0],table2.iloc[0,1],table2.iloc[0,2],table2.iloc[0,3]
    CSG0, CSGCC, CSGCD, CSGDC, CSGDD =table2.iloc[1,4],table2.iloc[1,0],table2.iloc[1,1],table2.iloc[1,2],table2.iloc[1,3]
    DSG0, DSGCC, DSGCD, DSGDC, DSGDD = table2.iloc[2,4],table2.iloc[2,0],table2.iloc[2,1],table2.iloc[2,2],table2.iloc[2,3]
    
    return(AD0, ADCC, ADCD, ADDC, ADDD
          ,CSG0, CSGCC, CSGCD, CSGDC, CSGDD
          ,DSG0, DSGCC, DSGCD, DSGDC, DSGDD)

# Main

In [12]:
def Main_MLE_K3 (data,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB, CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB):
    
    class1 = []
    class2 = []
    class3 = []
    MLE = []
    class_change = []
    
    
    
    for i in range (1) :

        if i >0 :
            K3 = MLE_K3 (data,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB,CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB)
            table_K3 = MLE_calculate (K3)
            mle , table_K3 = MLE_min(table_K3)
            df_cluster_K3 = Cluster (table_K3)
            fitt = change_class(df_cluster_K3k3,df_cluster_K3,fit=0)
            df_vlook_K3 = vLookup (K3, table_K3)
            class11,class22,class33 = N_class (df_vlook_K3)
        
            
        else:
            K3 = MLE_K3 (data,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB,CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB)
            table_K3 = MLE_calculate (K3)
            mle , table_K3 = MLE_min(table_K3)
            df_cluster_K3 = Cluster (table_K3)
            df_vlook_K3 = vLookup (K3, table_K3)
            class11,class22,class33 = N_class (df_vlook_K3)
            fitt = 1
            df_cluster_K3k3 = df_cluster_K3
            
        
        AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB,CSG0BB, CSGCCBB, CSGCDBB, CSGDCBB, CSGDDBB,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB = vector(df_vlook_K3) 
          
        
        class_change.append(fitt)
        MLE.append(mle)
        class1.append(class11)
        class2.append(class22)
        class3.append(class33)
    
    
    return(MLE, class1,class2,class3,class_change,AD0BB, ADCCBB, ADCDBB, ADDCBB, ADDDBB ,CSG0BB, CSGCCBB, CSGCDBB, CSGDCBB, CSGDDBB ,DSG0BB, DSGCCBB, DSGCDBB, DSGDCBB, DSGDDBB)



# Analysis

In [ ]:
MLE, class11,class22,class33,class_change,AD0, ADCC, ADCD, ADDC, ADDD ,CSG0, CSGCC, CSGCD, CSGDC, CSGDD ,DSG0, DSGCC, DSGCD, DSGDC, DSGDD = Main_MLE_K3(df,AD0, ADCC, ADCD, ADDC, ADDD ,CSG0, CSGCC, CSGCD, CSGDC, CSGDD ,DSG0, DSGCC, DSGCD, DSGDC, DSGDD)

In [ ]:
print (class11)

print (class22)

print (class33)

print (class_change)

print(MLE)

In [ ]:
MLE_Dd = pd.DataFrame([MLE_D, class11_B,class22_B,class33_B])
MLE_Dd.to_excel("bb.xlsx")